In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [ ]:
text = "The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that."

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("xsum")

In [ ]:
text = dataset['train'][0]['document']

'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but

# New Section

In [ ]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')
from google.colab import files
sentences=sent_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create a folder in the root directory
!mkdir -p "/content/drive/My Drive/Textranked BBC News/sport"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for x in range(401):
  x = x+1
  if x < 10 :
    file_name = "00" + str(x) + ".txt"
  if 9 < x < 100 :
    file_name = "0" + str(x) + ".txt"
  if x > 99 :
    file_name = str(x) + ".txt"
  !rm {file_name}

In [ ]:
n = 0
for n in range(511):
  n = n+1
  if n < 10 :
    file_name = "00" + str(n) + ".txt"
    new_file_name = "textrank_00" + str(n) + ".txt"
    file_name_directory = "/content/drive/My Drive/Textranked BBC News/sport/" + new_file_name
  if 9 < n < 100 :
    file_name = "0" + str(n) + ".txt"
    new_file_name = "textrank_0" + str(n) + ".txt"
    file_name_directory = "/content/drive/My Drive/Textranked BBC News/sport/" + new_file_name
  if n > 99 :
    file_name = str(n) + ".txt"
    new_file_name = "textrank_" + str(n) + ".txt"
    file_name_directory ="/content/drive/My Drive/Textranked BBC News/sport/" + new_file_name
  with open(file_name, encoding="utf-8") as m:
    hypo = [line.rstrip('\n') for line in m]
  for h in hypo:
    if h=='' :
      hypo.remove(h)
  del hypo[0]
  text = ' '.join(hypo)
  sentences=sent_tokenize(text)
  sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
  stop_words = stopwords.words('english')
  sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

  w2v=Word2Vec(sentence_tokens,vector_size=3,min_count=1,epochs=1000)
  sentence_embeddings=[[w2v.wv[word][0] for word in words] for words in sentence_tokens]
  max_len=max([len(tokens) for tokens in sentence_tokens])
  sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

  similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
  for i,row_embedding in enumerate(sentence_embeddings):
      for j,column_embedding in enumerate(sentence_embeddings):
          similarity_matrix[i][j]=abs(1-spatial.distance.cosine(row_embedding,column_embedding))
  nx_graph = nx.from_numpy_array(similarity_matrix)
  scores = nx.pagerank(nx_graph)
  top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
  top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
  new_file_name = "/content/new_folder/" + new_file_name
  for sent in sentences:
    if sent in top.keys():
      #print(sent+'\n')
      #print(new_file_name)
      #print(file_name_directory)
      with open(file_name_directory, 'a') as f:
        f.write(sent)
  print("text " + str(n) + " summarized")
  #files.download(file_name_directory)



In [ ]:
!ls "/content/drive/My Drive/Textranked BBC News/tech"

In [ ]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
print(similarity_matrix)
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=abs(1-spatial.distance.cosine(row_embedding,column_embedding)) #!!!!!!!!!!!! please check negative distance
print(similarity_matrix)

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1.         0.52101141 0.13005024 0.37065312 0.01869271 0.35524154
  0.20151387 0.38051304 0.24294108]
 [0.52101141 1.         0.44421318 0.34088176 0.32418856 0.54517913
  0.53458232 0.03019713 0.50487602]
 [0.13005024 0.44421318 1.         0.7473821  0.82897246 0.60159701
  0.73471171 0.66323185 0.84917402]
 [0.37065312 0.34088176 0.7473821  1.         0.67390656 0.69016725
  0.76443368 0.43235549 0.72629368]
 [0.01869271 0.32418856 0.82897246 0.67390656 1.         0.49716294
  0.64910191 0.78465319 0.90690953]
 [0.35524154 0.54517913 0.60159701 0.69016725 0.49716294 1.
  0.92602736 0.23000433 0.637802  ]
 [0.20151387 0.53458232 0.73471171 0.76443368 0.64910191 0.92602736
  1.         0.38033426 0.72447163]
 [0.38051